# Attack Exploring

Exploring the Attack and setting the right Parameters

## Imports

### GPU Information

The following GPU is used while Training the Model. Google Colab is used for Training so the GPU might change while running it later.

**GPU-Specifications:**
*   Name: Tesla P100-PCIE
*   GPU Memory: 16280MiB

In [ ]:
!nvidia-smi

Mon Aug 23 20:19:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Importing Data
Files are uploaded in Google Drive. Notebook is connected to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Modulation-Classification/RadioML-Dataset/Data/PGD-Attack

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Modulation-Classification/RadioML-Dataset/Data/PGD-Attack


### Importing Libraries

Cleverhans is used for generating Adversarial Examples.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pimg
import seaborn as sns
import scipy.io
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score

# Tensorflow Libraries
!pip3 install tensorflow==2.2.0
!pip3 install keras-tuner==1.0.2
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
from tensorflow.keras.layers import Dense, Flatten, ReLU, Conv2D, Input, MaxPooling2D, BatchNormalization, AveragePooling2D, Reshape, ZeroPadding2D, Dropout, Activation, Permute, Concatenate
from tensorflow.keras.models import Model,Sequential

# ART Libraries
"""
!pip3 install adversarial-robustness-toolbox[tensorflow]
from art.attacks.evasion import CarliniL2Method
from art.estimators.classification import KerasClassifier
"""

# Cleverhans Libraries
!pip3 install cleverhans
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.tf2.attacks.carlini_wagner_l2 import CarliniWagnerL2

### Importing Model

In [ ]:
# Model
Robust_CNN = tf.keras.models.load_model("../Model/Clean_Robust_CNN.h5")

### Importing Dataset

In [ ]:
# Loading Training Data
Clean_X_Train = np.load("../Clean-Data/Clean_X_Train.npy")
Clean_y_Train = np.load("../Clean-Data/Clean_y_Train.npy")

Clean_X_Train = np.expand_dims(np.transpose(Clean_X_Train, (0,2,1)), axis=-1)

## Designing Attack

Modulation Schemes

In [ ]:
f = open("../Dataset/ModulationMap.pkl", "rb")
ModulationMap = pickle.load(f)
ModulationMap = dict((v,k) for k,v in ModulationMap.items())
ModulationMap

{'8PSK': 0,
 'AM-DSB': 1,
 'AM-SSB': 2,
 'BPSK': 3,
 'CPFSK': 4,
 'GFSK': 5,
 'PAM4': 6,
 'QAM16': 7,
 'QAM64': 8,
 'QPSK': 9,
 'WBFM': 10}

Attack Settings

In [ ]:
# Parameters of PGD/FGM
Epsilon = 0.1
Epsilon_Step = 0.005
nb_Iterations = 200
Max = np.max(Clean_X_Train)
Min = np.min(Clean_X_Train)

Exploring Parameters

In [ ]:
Robust_CNN.evaluate(Clean_X_Train, Clean_y_Train)

5500/5500 [==============================] - 12s 2ms/step - loss: 1.0724 - accuracy: 0.6074


[1.0723711252212524, 0.6073636412620544]

FGM Attack

In [ ]:
for i in range(0,Clean_X_Train.shape[0],16000):

    Adv = fast_gradient_method(
        model_fn = Robust_CNN,
        x = Clean_X_Train[i+16000-4096:i+16000],
        eps = Epsilon,
        norm = np.inf,
        clip_min = Min,
        clip_max = Max,
        targeted=False,
        y = np.argmax(Clean_y_Train[i+16000-4096:i+16000], axis=1)
    )

    y_True = np.argmax(Clean_y_Train[i+16000-4096:i+16000], axis=1)
    y_Pred = np.argmax(Robust_CNN.predict(Clean_X_Train[i:i+4096]), axis=1)
    y_Adv_Pred = np.argmax(Robust_CNN.predict(Adv), axis=1)

    print ("i = " + str(i) +  str(", True Labels = ") + str(y_True) + str(", Predicted Adv Labels = ") + str(y_Adv_Pred) + str(", Accuracy of Attack = ") + str(1-accuracy_score(y_True,y_Adv_Pred)))

i = 0, True Labels = [0 0 0 ... 0 0 0], Predicted Adv Labels = [8 6 8 ... 8 8 6], Accuracy of Attack = 1.0
i = 16000, True Labels = [1 1 1 ... 1 1 1], Predicted Adv Labels = [6 6 6 ... 6 6 6], Accuracy of Attack = 0.99267578125
i = 32000, True Labels = [2 2 2 ... 2 2 2], Predicted Adv Labels = [8 6 6 ... 8 8 6], Accuracy of Attack = 1.0
i = 48000, True Labels = [3 3 3 ... 3 3 3], Predicted Adv Labels = [6 6 8 ... 6 6 6], Accuracy of Attack = 1.0
i = 64000, True Labels = [4 4 4 ... 4 4 4], Predicted Adv Labels = [6 6 8 ... 8 8 8], Accuracy of Attack = 1.0
i = 80000, True Labels = [5 5 5 ... 5 5 5], Predicted Adv Labels = [8 6 8 ... 8 6 8], Accuracy of Attack = 1.0
i = 96000, True Labels = [6 6 6 ... 6 6 6], Predicted Adv Labels = [6 6 6 ... 8 6 6], Accuracy of Attack = 0.501220703125
i = 112000, True Labels = [7 7 7 ... 7 7 7], Predicted Adv Labels = [8 8 6 ... 8 6 6], Accuracy of Attack = 1.0
i = 128000, True Labels = [8 8 8 ... 8 8 8], Predicted Adv Labels = [8 6 8 ... 8 8 8], Accurac

PGD Attack

In [ ]:
for i in range(0,Clean_X_Train.shape[0],16000):

    Adv = projected_gradient_descent(
            model_fn = Robust_CNN,
            x = Clean_X_Train[i+16000-4096:i+16000],
            eps = Epsilon,
            eps_iter = Epsilon_Step,
            nb_iter = nb_Iterations,
            norm = np.inf,
            clip_min = Min,
            clip_max = Max,
            targeted=False,
            y = np.argmax(Clean_y_Train[i+16000-4096:i+16000], axis=1)
        )

    y_True = np.argmax(Clean_y_Train[i+16000-4096:i+16000], axis=1)
    y_Pred = np.argmax(Robust_CNN.predict(Clean_X_Train[i:i+4096]), axis=1)
    y_Adv_Pred = np.argmax(Robust_CNN.predict(Adv), axis=1)

    print ("i = " + str(i) +  str(", True Labels = ") + str(y_True) + str(", Predicted Adv Labels = ") + str(y_Adv_Pred) + str(", Accuracy of Attack = ") + str(1-accuracy_score(y_True,y_Adv_Pred)))

i = 0, True Labels = [0 0 0 ... 0 0 0], Predicted Adv Labels = [6 6 8 ... 8 6 6], Accuracy of Attack = 1.0
i = 16000, True Labels = [1 1 1 ... 1 1 1], Predicted Adv Labels = [ 8 10  6 ...  6  6  6], Accuracy of Attack = 1.0
i = 32000, True Labels = [2 2 2 ... 2 2 2], Predicted Adv Labels = [8 8 6 ... 8 8 8], Accuracy of Attack = 1.0
i = 48000, True Labels = [3 3 3 ... 3 3 3], Predicted Adv Labels = [6 8 6 ... 6 8 6], Accuracy of Attack = 1.0
i = 64000, True Labels = [4 4 4 ... 4 4 4], Predicted Adv Labels = [8 8 8 ... 8 8 8], Accuracy of Attack = 1.0
i = 80000, True Labels = [5 5 5 ... 5 5 5], Predicted Adv Labels = [8 8 8 ... 8 8 8], Accuracy of Attack = 1.0
i = 96000, True Labels = [6 6 6 ... 6 6 6], Predicted Adv Labels = [8 8 8 ... 8 8 8], Accuracy of Attack = 1.0
i = 112000, True Labels = [7 7 7 ... 7 7 7], Predicted Adv Labels = [8 6 8 ... 8 8 8], Accuracy of Attack = 1.0
i = 128000, True Labels = [8 8 8 ... 8 8 8], Predicted Adv Labels = [6 6 7 ... 3 6 6], Accuracy of Attack = 1